In [39]:
# Importing libraries

import langchain
import pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
import langchain_pinecone
from langchain_pinecone import PineconeEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI




In [40]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env.local')


True

In [41]:
import os

In [42]:
## Read documents

def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents


In [43]:
doc = read_doc("documents/")
len(doc)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000021BF9268B60>


8

In [44]:
## Divide the documents into chunks

def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    doc = text_splitter.split_documents(docs)
    return doc
           

In [45]:
documents = chunk_data(docs=doc)
documents

[Document(page_content="AWS\nLingo\nand\nExplanations\n1.\nVirtualization\n:\nThis\nis\nlike\ncreating\na\npretend\ncomputer\nversion\ninside\na\nreal\none.\nIt's\nlike\nhaving\na\ncomputer\ninside\nyour\ncomputer.\nCloud\nproviders,\nincluding\nAWS,\nuse\nvirtualization\nto\ncreate\ninstances\nthat\nyou\ncan\nuse\njust\nlike\nreal\ncomputers.\n2.\nElasticity\n:\nThink\nof\nthis\nas\nthe\ncloud's\nability\nto\nstretch\nand\nshrink\nbased\non\ndemand.\nIt's\nlike\nyour\nroom\nautomatically\ngetting\nbigger\nwhen\nyou\nhave\nfriends\nover,\nand\nsmaller\nwhen\nthey\nleave.\nAWS\nallows\nyou\nto\nscale\nyour\nresources\nup\nor\ndown\nas\nneeded.\n3.\nLoad\nBalancing\n:\nImagine\nyou're\nat\na\nbuffet\nwith\nmultiple\nserving\nstations.\nLoad\nbalancing\nis\nlike\nmaking\nsure\neveryone\ngets\ntheir\nfood\nquickly\nby\nsending\nthem\nto\nthe\nless\ncrowded\nstations.\nIn\nAWS,\nload\nbalancers\ndistribute\nincoming\ntraffic\nevenly\namong\nyour", metadata={'source': 'documents\\AWS_stuff.p

In [46]:
# Embeddings 
model_name = "multilingual-e5-large"  
pc_api_key = os.environ.get("PINECONE_API_KEY")
embeddings = PineconeEmbeddings(  
    model=model_name,  
    pinecone_api_key=pc_api_key  
)  


In [47]:
# Test vectors

vectors = embeddings.embed_query("How are you")
print(vectors)
len(vectors)

[0.03741455078125, -0.017791748046875, -0.0164642333984375, -0.028961181640625, 0.036834716796875, -0.042205810546875, 0.0028781890869140625, 0.043060302734375, 0.04278564453125, -0.0259552001953125, 0.0167388916015625, 0.034332275390625, -0.06591796875, -0.05511474609375, -0.01003265380859375, -0.027374267578125, -0.0129547119140625, 0.0294952392578125, -0.0011644363403320312, -0.009490966796875, 0.00829315185546875, -5.3048133850097656e-05, -0.009063720703125, -0.03375244140625, 0.00015854835510253906, 0.0179901123046875, -0.0369873046875, -0.0142059326171875, -0.00701904296875, -0.042236328125, -0.01031494140625, 0.0217132568359375, -0.0204315185546875, -0.01349639892578125, -0.02734375, 0.041473388671875, 0.01197052001953125, 0.0207672119140625, -0.034454345703125, 0.0280914306640625, -0.04254150390625, 0.028717041015625, -0.005527496337890625, -0.034454345703125, -0.0030307769775390625, 0.019775390625, 0.0028934478759765625, 0.0007085800170898438, -0.00589752197265625, 0.023666381

1024

In [48]:
# Vector search Database

index_name = os.environ.get("PINECONE_INDEX_NAME")

pc_doc_search = PineconeVectorStore.from_documents(
    documents=doc,
    index_name=index_name,
    embedding=embeddings, 
    namespace="wondervector5000" 
)


In [49]:
# Cosine Retrieval from Vector Database

def retrieve_query(query, k=2):
    matching_results = pc_doc_search.similarity_search(query, k=k)
    return matching_results

In [57]:
from langchain.chains.question_answering import load_qa_chain

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

chat_model_name = "gpt-3.5-turbo-instruct-0914"
llm = OpenAI(api_key=OPENAI_API_KEY, model_name=chat_model_name, temperature=0.5)
chain = load_qa_chain(llm, chain_type="stuff")

In [58]:
# Search answers from Vector Database

def retrieve_answers(query):
    doc_search = retrieve_query(query)
    # print(doc_search)
    response = chain.invoke({"input_documents": doc_search, "question": query})
    return response
    
    


In [85]:
first_query = "what is an apple?"
second_query = "what is an LPU?"
third_query = "what is serverless?"
fourth_query = "Can AWS and groq be used to create AI applications?"
answer1 = retrieve_answers(first_query)
answer2 = retrieve_answers(second_query)
answer3 = retrieve_answers(third_query)
answer4 = retrieve_answers(fourth_query)

print("First question:", answer1.get("output_text"), "")
print("Second question:", answer2.get("output_text"), "")
print("Third question:", answer3.get("output_text"), "")
print("Fourth question:", answer4.get("output_text"), "")


First question:  I don't know. This context does not mention anything about apples. 
Second question:  An LPU, or Language Processing Unit, is a type of processor designed specifically for AI and capable of running large language models at high speeds and energy efficiency. It is a new category of processor created by Groq. 
Third question:  Serverless is a term used to describe a cloud computing model where the cloud provider manages the infrastructure and servers, allowing developers to focus solely on writing and deploying code. In AWS, this is achieved through services like AWS Lambda. 
Fourth question:  Yes, AWS and Groq can be used together to create AI applications. Groq's LPU technology can be used to optimize and accelerate AI inference, while AWS provides a cloud computing platform for hosting and managing AI applications. 
